# <b> Clean up </b>

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## Clean up

In [2]:
import boto3
from utils.ddb import ddb_handler
from utils.iam import iam_handler
from utils.ssm import parameter_store
from utils.lambda_func import lambda_handler
from utils.api_gateway import api_gateway_handler

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

> API-Gateway and Lambda

In [4]:
lam_handler = lambda_handler(region_name=pm.get_params(key="".join([strPrefix, "REGION"])))
apig = api_gateway_handler(pm.get_params(key=strPrefix+"REGION"))
strLambdaName = pm.get_params(key=strPrefix+"LAMBDA-NAME")
strApiId = pm.get_params(key=strPrefix+"API-ID")

lam_handler.delete_function(strLambdaName)
apig._delete_rest_api(api_id=strApiId)

  lambda function: [API-GATEWAY-LAMBDA-DEMO-Lambda] is deleted successfully
  api-gateway: [eqza38p6m9] is deleted successfully


> DynamoDB and RDS (Aurora)

In [6]:
ddb_inf_res = ddb_handler(strTableName=pm.get_params(strPrefix+"INFERENCE-RES-TABLE-NAME"), region_name=pm.get_params(key=strPrefix+"REGION"))
ddb_inf_res.delete_table()

Table [API-GATEWAY-LAMBDA-DEMO-inference-results] is ACTIVE now.
Table [API-GATEWAY-LAMBDA-DEMO-inference-results] is deleted successfully.


>Role

In [8]:
iam = iam_handler()
listRoleNames = [strRoleName for strRoleName in iam.get_all_role_names() if strPrefix in strRoleName]
for strRoleName in listRoleNames:
    print (strRoleName)
    iam.delete_role(strRoleName=strRoleName)

>Parameter store

In [15]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]
pm.delete_param(listParams)

  parameters: ['PREFIX'] is deleted successfully
